In [1]:
import pandas as pd
import numpy as np

In [2]:
#Read Data and Cleaning

ds = pd.read_csv('../datasets/hqa_new.csv', sep = ';', skiprows = 4, nrows=180, header = 1)
ds.columns = ds.columns.str.strip()
ds.set_index('Variables',inplace = True)
ds.index.str.strip()
ds = ds.applymap(lambda x: int(x.replace(' ', '')))
ds.head(10)

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Variables,,,,,,,,,,,,,,,,,,,,,
Total,1622300,1679300,1730600,1768400,1816800,1860900,1916400,1969700,2020800,2067300,...,2523000,2576000,2601900,2626400,2669100,2732600,2750900,2795200,2846000,2877000
25 - 29 Years,281500,272100,271400,255300,256100,239900,240700,248300,242200,251500,...,237800,249400,230900,223200,223700,231700,232800,253000,242300,251800
30 - 34 Years,292700,288300,292200,283400,291000,292400,298200,293000,290000,293400,...,280200,289300,280100,271700,274600,276900,269900,268200,262300,263800
35 - 39 Years,252600,259900,266700,278300,291500,301200,305800,303400,307700,309900,...,312900,317300,306900,302300,297500,287900,300200,288900,305700,303400
40 - 44 Years,203900,226500,232100,248900,248300,264400,278700,286200,299600,300300,...,311900,303500,302300,310800,310300,305100,315300,304000,320000,313100
45 - 49 Years,127500,137700,155100,175700,190400,211300,233900,244500,252200,251800,...,319000,319600,321700,319200,315100,298600,301500,301100,313700,313200
50 - 54 Years,117500,136300,145000,141900,142200,132800,130900,154900,171400,186100,...,298400,303600,309600,313200,318900,318300,319400,321700,307800,316600
55 - 59 Years,99800,107200,108300,113700,118900,121700,127800,133000,132100,131900,...,233300,252800,267400,272100,280200,296400,296700,299500,285000,299000
60 - 64 Years,82800,89600,93200,95100,96000,97200,95600,99100,102000,113400,...,182100,196400,209400,219700,229400,246200,248200,261400,261700,274800


In [401]:
#Data for 2018 & droping per age

#function to convert column values to % values 
def col_to_perc(df_column):
    return df_column / df_column.sum() * 100

#Qualifications ordered from lowest to highest
sorted_level_list = ['Below Secondary', 'Secondary', 'Post Secondary (Non-Tertiary)', 
                     'Diploma & Professional Qualification', 'University']

# New df with only 2018 data
ds_2018 = ds [['2018']]

ds_2018 = ds_2018.loc[[st.startswith('Total') for st in ds_2018.index]] # Droping Per Age Rows
ds_2018.reset_index(inplace = True) # Passing index to column

#Creating column with Education Level Info Only
ds_2018['Education Level'] = ds_2018['Variables'].str.lstrip('Total - ').str.lstrip('Males - ').str.lstrip('Females - ')

#Creating column with Gender Only
ds_2018 = ds_2018.loc[6:17] # Drop Totals Lines, not needed
ds_2018.loc[6:11,'Gender'] = 'Male'
ds_2018.loc[12:17,'Gender'] = 'Female'
ds_2018.drop(['Variables'], axis =1, inplace = True) #Droping agg string no need anymore

# Pivot Gender to Columns and agg per values for 2018
ds_2018 = pd.pivot_table(ds_2018, values='2018', index=['Education Level'], columns=['Gender'], aggfunc=np.sum)
ds_2018.drop('',axis = 0, inplace = True) # Drop totals rows, not need

ds_2018 = ds_2018.reindex(sorted_level_list) #re-indexing to sort rows by qualf. from lowest to highest

# Convert columns to % 
ds_2018 ['Female'] = col_to_perc (ds_2018['Female']) 
ds_2018 ['Male'] = col_to_perc (ds_2018['Male']) 

ds_2018.loc["Total"] = ds_2018.sum() # Get Totals per Gender
ds_2018 = ds_2018.round(1) # round to 1 decimal plate the whole df
ds_2018

Gender,Female,Male
Education Level,,
Below Secondary,29.4,23.2
Secondary,19.0,16.4
Post Secondary (Non-Tertiary),7.8,10.4
Diploma & Professional Qualification,14.4,15.9
University,29.4,34.0
Total,100.0,100.0


In [416]:
# Analysis per decade

# aggregate data per decade - 90s', 00's and 10's
ds['1990 - 1999'] = ds.loc[:,'1990':'1999'].sum(axis=1)
ds['2000 - 2009'] = ds.loc[:,'2000':'2009'].sum(axis=1)
ds['2010 - 2018'] = ds.loc[:,'2010':'2018'].sum(axis=1)

#create new df with only per decade data
ds_decade = ds.loc[:,'1990 - 1999':]

ds_decade = ds_decade.loc[[st.startswith('Total') for st in ds_decade.index]] # Droping Per Age Rows
ds_decade.reset_index(inplace = True) # Passing index to column

#Creating column with Education Level Info Only
ds_decade['Education Level'] = ds_decade['Variables'].str.lstrip('Total - ').str.lstrip('Males - ').str.lstrip('Females - ')
                      
#Creating column with Gender Only
ds_decade = ds_decade.loc[6:17] # Drop Totals Lines, not needed
ds_decade.loc[6:11,'Gender'] = 'Male'
ds_decade.loc[12:17,'Gender'] = 'Female'
ds_decade.drop(['Variables'], axis =1, inplace = True) #Droping agg string no need anymore

# Pivot Gender to Columns and agg per values for each decade
ds_decade = pd.pivot_table(ds_decade, values=['1990 - 1999','2000 - 2009', '2010 - 2018'], index=['Education Level'], columns=['Gender'], aggfunc=np.sum)
ds_decade.drop('',axis = 0, inplace = True) # Drop totals rows, not need

ds_decade = ds_decade.reindex(sorted_level_list) #re-indexing to sort rows by qualf. from lowest to highest
ds_decade

# Convert columns to % 
ds_decade [('1990 - 1999', 'Female')] = col_to_perc(ds_decade [('1990 - 1999', 'Female')]) 
ds_decade [('1990 - 1999', 'Male')] = col_to_perc(ds_decade [('1990 - 1999', 'Male')] )
ds_decade [('2000 - 2009', 'Female')] = col_to_perc(ds_decade [('2000 - 2009', 'Female')])
ds_decade [('2000 - 2009', 'Male')] = col_to_perc(ds_decade [('2000 - 2009', 'Male')])
ds_decade [('2010 - 2018', 'Female')] = col_to_perc(ds_decade[('2010 - 2018', 'Female')])
ds_decade[('2010 - 2018', 'Male')] = col_to_perc(ds_decade [('2010 - 2018', 'Male')] )

ds_decade.loc["Total"] = ds_decade.sum(axis=0) # Get Totals per Gender

# Get the Ratio of % Female/ % Male in each Education Level per Decade
ds_decade [('1990 - 1999', 'F/M Ratio')] = ds_decade [('1990 - 1999', 'Female')] / ds_decade [('1990 - 1999', 'Male')]
ds_decade [('2000 - 2009', 'F/M Ratio')] = ds_decade [('2000 - 2009', 'Female')] / ds_decade [('2000 - 2009', 'Male')]
ds_decade [('2010 - 2018', 'F/M Ratio')] = ds_decade [('2010 - 2018', 'Female')] / ds_decade [('2010 - 2018', 'Male')]

ds_decade_final_perc.round(2)

1990 - 1999         2000 - 2009          \
Gender                                    Female    Male      Female    Male   
Education Level                                                                
Below Secondary                            58.26   52.58       44.83   39.14   
Secondary                                  24.96   23.52       23.77   21.50   
Post Secondary (Non-Tertiary)               6.44    6.43        6.49    7.27   
Diploma & Professional Qualification        4.23    7.35        9.55   11.79   
University                                  6.10   10.13       15.36   20.30   
Total                                     100.00  100.00      100.00  100.00   

                                     2010 - 2018         1990 - 1999  \
Gender                                    Female    Male   F/M Ratio   
Education Level                                                        
Below Secondary                            33.29   27.48        1.11   
Secondary                                  19.87   16.99        1.06   
Post Secondary (Non-Tertiary)               7.81   10.27        1.00   
Diploma & Professional Qualification       13.44   15.25        0.58   
University                                 25.60   30.02        0.60   
Total                                     100.00  100.00        1.00   

                                     2000 - 2009 2010 - 2018  
Gender                                 F/M Ratio   F/M Ratio  
Education Level                                               
Below Secondary                             1.15        1.21  
Secondary                                   1.11        1.17  
Post Secondary (Non-Tertiary)               0.89        0.76  
Diploma & Professional Qualification        0.81        0.88  
University                                  0.76        0.85  
Total                                       1.00        1.00

In [ ]:
# Insights

# 1) There is still gender inequality in acess to the highest degree of Education tracked (University) in 2018, 
# as the % of men attaining this degree is 4.6% above the women %.

# 2) Per decade evolution shows:
# 2.1) For both genders there is a substantial increase in Education Level from 90 to 00 and from 00 to 10s.
# 2.2) The F/M Ratio shows that, in spite of still existing the gender gap is closing on a decade-to-decade analysis.
# when looking at the highest degree of qualifications
# 2.3) However, the gap is widening when we look at the the lowest degree of qualifications for the same periods (Below Secondary)
# / Secondary